# TextToSpeech 文本转语音

TextToSpeech组件为Panel带来文本转语音功能，它封装了[HTML5 SpeechSynthesis API](https://developer.mozilla.org/en-US/docs/Web/API/SpeechSynthesis)和[HTML SpeechSynthesisUtterance API](https://developer.mozilla.org/en-US/docs/Web/API/SpeechSynthesisUtterance)。

底层实现为`panel.widgets.TextToSpeech`，参数基本一致，参考文档：https://panel.holoviz.org/reference/widgets/TextToSpeech.html


In [ ]:
##ignore
%load_ext vuepy
from panel_vuepy import vpanel


## 基本用法

文本转语音组件可以将文本转换为语音并播放出来。请注意，该组件本身在视觉上不显示任何内容，但仍需添加到应用程序中才能使用。


In [ ]:
%%vuepy_run --plugins vpanel --show-code
<template>
  <PnCol>
    <PnTextToSpeech 
      name="语音合成" 
      value="你好，欢迎使用Panel的文本转语音组件。"
      @change="on_change"
    />
    <PnButton 
      label="点击播放" 
      button_type="primary" 
      @click="speak()"
    />
  </PnCol>
</template>
<script lang='py'>
import panel as pn
from vuepy import ref

text_to_speech_ref = ref(None)

def speak():
    text_to_speech_ref.value.speak = True
    
def on_change(event):
    text_to_speech_ref.value = event['owner']
    print(f"语音状态变化: {event}")
</script>


## 自动播放

当`auto_speak`设置为true时（默认值），每当`value`更改时，都会自动播放语音。


In [ ]:
%%vuepy_run --plugins vpanel --show-code
<template>
  <PnCol>
    <PnTextToSpeech 
      name="自动播放" 
      :value="text.value"
      :auto_speak="True"
      ref="tts"
    />
    <PnTextAreaInput 
      v-model="text.value"
      rows="3"
      placeholder="输入文本，修改后会自动播放"
    />
    <PnRow>
      <PnButton label="暂停" @click="pause()" />
      <PnButton label="恢复" @click="resume()" />
      <PnButton label="取消" @click="cancel()" />
    </PnRow>
  </PnCol>
</template>
<script lang='py'>
import panel as pn
from vuepy import ref

text = ref("输入文本，修改后会自动播放")

def pause():
    tts = pn.state.curdoc.select_one({'name': '自动播放'})
    if tts:
        tts.pause = True

def resume():
    tts = pn.state.curdoc.select_one({'name': '自动播放'})
    if tts:
        tts.resume = True
        
def cancel():
    tts = pn.state.curdoc.select_one({'name': '自动播放'})
    if tts:
        tts.cancel = True
</script>


## 语音参数调整

可以通过设置`lang`、`pitch`、`rate`和`volume`参数来调整语音的特性。


In [ ]:
%%vuepy_run --plugins vpanel --show-code
<template>
  <PnCol>
    <PnTextToSpeech 
      name="语音参数调整" 
      value="这是一段用于测试语音参数的文本。"
      :auto_speak="False"
      :lang="lang.value"
      :pitch="pitch.value"
      :rate="rate.value"
      :volume="volume.value"
      ref="tts_custom"
    />
    
    <PnRow>
      <PnCol>
        <PnStaticText value="语言:" />
        <PnSelect 
          v-model="lang.value"
          :options="['zh-CN', 'en-US', 'ja-JP', 'fr-FR', 'de-DE']"
        />
      </PnCol>
      
      <PnCol>
        <PnStaticText value="音调:" />
        <PnFloatSlider 
          v-model="pitch.value"
          :start="0"
          :end="2"
          :step="0.1"
        />
      </PnCol>
      
      <PnCol>
        <PnStaticText value="语速:" />
        <PnFloatSlider 
          v-model="rate.value"
          :start="0.1"
          :end="2"
          :step="0.1"
        />
      </PnCol>
      
      <PnCol>
        <PnStaticText value="音量:" />
        <PnFloatSlider 
          v-model="volume.value"
          :start="0"
          :end="1"
          :step="0.1"
        />
      </PnCol>
    </PnRow>
    
    <PnButton 
      label="播放" 
      button_type="primary" 
      @click="speak_custom()"
    />
  </PnCol>
</template>
<script lang='py'>
import panel as pn
from vuepy import ref

lang = ref("zh-CN")
pitch = ref(1.0)
rate = ref(1.0)
volume = ref(1.0)

def speak_custom():
    tts = pn.state.curdoc.select_one({'name': '语音参数调整'})
    if tts:
        tts.speak = True
</script>


## 长文本示例

TextToSpeech组件可以处理较长的文本内容。


In [ ]:
%%vuepy_run --plugins vpanel --show-code
<template>
  <PnCol>
    <PnTextToSpeech 
      name="长文本示例" 
      :value="LONG_TEXT"
      :auto_speak="False"
      ref="tts_long"
    />
    
    <PnButton 
      label="播放长文本" 
      button_type="success" 
      @click="speak_long()"
    />
    
    <PnTextAreaInput 
      :value="LONG_TEXT"
      rows="10"
      disabled
    />
  </PnCol>
</template>
<script lang='py'>
import panel as pn
from vuepy import ref

LONG_TEXT = """伊索寓言 - 猴王和狐狸

据说有一段时期，所有的动物都和谐相处。狮子不追逐牛，狼不捕猎羊，猫头鹰不俯冲捕捉田野里的老鼠。

每年他们会聚在一起选择一位国王，他将在接下来的十二个月统治动物王国。那些想要当国王的动物会主动站出来，发表演讲并展示他们的实力或智慧。然后所有聚集在一起的动物会投票，得票最多的动物将被加冕为王。这可能就是人类选举的起源！

猴子很清楚自己既不强壮也不聪明，演讲能力也不出众，但是，天啊，它跳舞可厉害了！所以它做了自己最擅长的事情，它跳得杂技般且充满活力，做出巨大的跳跃、后空翻和侧手翻，真正让观众眼花缭乱。相比之下，大象显得严肃笨重，狮子强大专制，蛇狡猾且阴险。

在场的没有人确切记得事情是如何发生的，但不知怎么的，猴子以明显的多数票获胜，被宣布为来年的动物王国之王。大多数动物对这个结果似乎相当满意，因为他们知道猴子不会太认真对待自己的职责，也不会对他们提出繁重的要求，或要求太多正式的服从表示。但也有一些动物认为选举猴子降低了王位的地位，狐狸就是其中之一；事实上，狐狸相当厌恶这个结果，而且它不在乎谁知道这一点。于是它开始策划一个计划让猴子看起来很愚蠢。

它从森林里收集了一些新鲜的水果，芒果、无花果和枣子，把它们放在它找到的一个陷阱上。它等着猴子经过，然后对它喊道："陛下，看看我在路边发现的这些美味的小食。我本想大吃一顿，但我想起水果是您最喜欢的食物，我觉得应该为您，我们敬爱的国王，留着它们！"

猴子既抵挡不住奉承，也抵挡不住水果的诱惑，只好勉强控制住自己，匆匆低声说了句"谢谢你，狐狸先生"，然后直奔水果而去。"唰"的一声，"咔嗒"一声，陷阱夹住了，我们不幸的猴子王"呀呀"大叫，陷阱紧紧夹住了它的爪子。

猴子痛苦地责备狐狸把它引入这样危险的境地，但狐狸只是一个劲地笑。"你自称是所有动物之王，"它喊道，"却如此轻易就上当受骗！"

伊索"""

def speak_long():
    tts = pn.state.curdoc.select_one({'name': '长文本示例'})
    if tts:
        tts.speak = True
</script>


## API

### 属性

| 属性名 | 说明 | 类型 | 默认值 |
| -------- | ------------------- | ---------------------------------------------------------------| ------- |
| value | 将在发声时合成的文本 | ^[string] | — |
| auto_speak | 值更改时是否自动发声 | ^[boolean] | true |
| lang | 语音的语言 | ^[string] | — |
| voice | 用于发声的语音 | ^[SpeechSynthesisVoice] | — |
| pitch | 语音的音调，0到2之间的数字 | ^[float] | 1 |
| rate | 语音的速度，0.1到10之间的数字 | ^[float] | 1 |
| volume | 语音的音量，0到1之间的数字 | ^[float] | 1 |
| speak | 发声动作 | ^[boolean] | false |
| cancel | 取消所有待发声的语音 | ^[boolean] | false |
| pause | 暂停语音合成 | ^[boolean] | false |
| resume | 恢复语音合成 | ^[boolean] | false |
| voices | 当前设备上可用的所有语音列表 | ^[List[Voice]] | [] |
| paused | 语音合成是否处于暂停状态 | ^[boolean] | false |
| pending | 语音队列中是否有尚未发声的语音 | ^[boolean] | false |
| speaking | 当前是否正在发声 | ^[boolean] | false |
| name | 组件标题 | ^[string] | — |

### Events

| 事件名 | 说明 | 类型 |
| --- | --- | --- |
| change | 当组件状态改变时触发 | ^[Callable]`(event: dict) -> None` |

### Slots

| 插槽名 | 说明 |
| --- | --- |
| | |

### 方法

| 属性名 | 说明 | 类型 |
| --- | --- | --- |
| | | |


In [ ]:
##ignore
import panel as pn

pn.extension()

text_to_speech = pn.widgets.TextToSpeech(name="Text to Speech Widget", value="Hello Panel World", auto_speak=False)
text = pn.Param(text_to_speech.param.value, widgets={"value": {"widget_type": pn.widgets.TextAreaInput, "height": 300}})

pn.Row(text_to_speech.controls(jslink=False), text, text_to_speech, width=600) 